# Introduction to Simple RAG

Retrieval-Augmented Generation (RAG) is a hybrid approach that combines information retrieval with generative models. It enhances the performance of language models by incorporating external knowledge, which improves accuracy and factual correctness.

In a Simple RAG setup, we follow these steps:

1. **Data Ingestion**: Load and preprocess the text data.
2. **Chunking**: Break the data into smaller chunks to improve retrieval performance.
3. **Embedding Creation**: Convert the text chunks into numerical representations using an embedding model.
4. **Semantic Search**: Retrieve relevant chunks based on a user query.
5. **Response Generation**: Use a language model to generate a response based on retrieved text.

This notebook implements a Simple RAG approach, evaluates the model’s response, and explores various improvements.

## Setting Up the Environment
We begin by importing necessary libraries.

In [1]:
import fitz # PyMuPDF
import os
import numpy as np
import json
import google.generativeai as genai

In [2]:

import fitz
import os
import google.generativeai as genai
from dotenv import load_dotenv


## Extracting Text from a PDF File
To implement RAG, we first need a source of textual data. In this case, we extract text from a PDF file using the PyMuPDF library.

In [2]:
def extract_text_from_pdf(pdf_path):
    """
    Extracts text from a PDF file and prints the first `num_chars` characters.

    Args:
    pdf_path (str): Path to the PDF file.

    Returns:
    str: Extracted text from the PDF.
    """
    # Open the PDF file
    mypdf = fitz.open(pdf_path)
    all_text = ""  # Initialize an empty string to store the extracted text

    # Iterate through each page in the PDF
    for page_num in range(mypdf.page_count):
        page = mypdf[page_num]  # Get the page
        text = page.get_text("text")  # Extract text from the page
        all_text += text  # Append the extracted text to the all_text string

    return all_text  # Return the extracted text

## Chunking the Extracted Text
Once we have the extracted text, we divide it into smaller, overlapping chunks to improve retrieval accuracy.

In [3]:
def chunk_text(text, n, overlap):
    """
    Chunks the given text into segments of n characters with overlap.

    Args:
    text (str): The text to be chunked.
    n (int): The number of characters in each chunk.
    overlap (int): The number of overlapping characters between chunks.

    Returns:
    List[str]: A list of text chunks.
    """
    chunks = []  # Initialize an empty list to store the chunks
    
    # Loop through the text with a step size of (n - overlap)
    for i in range(0, len(text), n - overlap):
        # Append a chunk of text from index i to i + n to the chunks list
        chunks.append(text[i:i + n])

    return chunks  # Return the list of text chunks

## Setting Up the OpenAI API Client
We initialize the OpenAI client to generate embeddings and responses.

In [3]:

import fitz
import os
import google.generativeai as genai
from dotenv import load_dotenv


## Extracting and Chunking Text from a PDF File
Now, we load the PDF, extract text, and split it into chunks.

In [6]:
# Import the required library for PDF reading
from PyPDF2 import PdfReader

# --- Function Definitions ---

def extract_text_from_pdf(pdf_path):
    """Extracts text from all pages of a PDF file."""
    text = ""
    # Open the PDF file in binary read mode
    with open(pdf_path, 'rb') as file:
        pdf = PdfReader(file)
        # Loop through each page and extract text
        for page in pdf.pages:
            text += page.extract_text() or ""
    return text

def chunk_text(text, chunk_size, overlap):
    """Chunks text into segments with a specified overlap."""
    chunks = []
    start = 0
    # Loop through the text and create overlapping chunks
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        # Move the start position forward by the chunk size minus the overlap
        start += chunk_size - overlap
    return chunks

# --- Your Original Logic ---

# Define the path to the PDF file
# IMPORTANT: Make sure this path is correct on your system
pdf_path = "/Users/kekunkoya/Desktop/PHD/ISEM 770/Class Code SAT/AI_Information.pdf"

# Extract text from the PDF file by calling the function we defined
extracted_text = extract_text_from_pdf(pdf_path)

# Chunk the extracted text by calling the function we defined
text_chunks = chunk_text(extracted_text, 1000, 200)

# Print the number of text chunks created
print("Number of text chunks:", len(text_chunks))

# Print the first text chunk
print("\nFirst text chunk:")
# Print only the first chunk if it exists
if text_chunks:
    print(text_chunks[0])

Number of text chunks: 43

First text chunk:
Understanding Artificial Intelligence  
Chapter 1: Introduction to Artificial Intelligence  
Artificial intelligence (AI) refers to the ability of a digital computer or computer -controlled robot 
to perform tasks commonly associated with intelligent beings. The term is frequently applied to 
the project of developing systems endowed with the intelle ctual processes characteristic of 
humans, such as the ability to reason, discover meaning, generalize, or learn from past 
experience. Over the past few decades, advancements in computing power and data availability 
have significantly accelerated the develo pment and deployment of AI.  
Historical Context  
The idea of artificial intelligence has existed for centuries, often depicted in myths and fiction. 
However, the formal field of AI research began in the mid -20th century. The Dartmouth Workshop 
in 1956 is widely considered the birthplace of AI. Early AI r esearch focused on problem -sol

In [11]:
# --- Embeddings (Gemini) ---
def _embed_one(text: str, model: str = "models/embedding-001", retries: int = 3) -> List[float]:
    """Embed a single text with retry/backoff."""
    last_err = None
    for i in range(retries):
        try:
            resp = genai.embed_content(model=model, content=text)
            return resp["embedding"]
        except Exception as e:
            last_err = e
            # Friendly handling for common auth issues
            if "API key" in str(e) or "401" in str(e):
                raise RuntimeError("Google API auth failed: check GOOGLE_API_KEY") from e
            time.sleep(1.5 * (i + 1))
    raise last_err

def embed_texts(texts: List[str], model: str = "models/embedding-001") -> np.ndarray:
    """Embed a list of strings. Sequential on purpose (robust/simple)."""
    embs = []
    for t in texts:
        embs.append(_embed_one(t, model=model))
    return np.array(embs, dtype=np.float32)

# --- Main example ---
if __name__ == "__main__":
    # ...
    # 3) Embed chunks
    print("Embedding chunks with Gemini...")
    chunk_embs = embed_texts(chunks, model="models/embedding-001")
    
    # ...
    
    # 5) Example Q&A
    example_query = "What is AI according to this document?"
    q_vec = np.array(_embed_one(example_query, model="models/embedding-001"), dtype=np.float32)
    # ...

Embedding chunks with Gemini...


In [ ]:
# Import the required library for PDF reading
from PyPDF2 import PdfReader

# --- Function Definitions ---

def extract_text_from_pdf(pdf_path):
    """Extracts text from all pages of a PDF file."""
    text = ""
    # Open the PDF file in binary read mode
    with open(pdf_path, 'rb') as file:
        pdf = PdfReader(file)
        # Loop through each page and extract text
        for page in pdf.pages:
            text += page.extract_text() or ""
    return text

def chunk_text(text, chunk_size, overlap):
    """Chunks text into segments with a specified overlap."""
    chunks = []
    start = 0
    # Loop through the text and create overlapping chunks
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        # Move the start position forward by the chunk size minus the overlap
        start += chunk_size - overlap
    return chunks

# --- Your Original Logic ---

# Define the path to the PDF file
# IMPORTANT: Make sure this path is correct on your system
pdf_path = "/Users/kekunkoya/Desktop/PHD/ISEM 770/Class Code SAT/AI_Information.pdf"

# Extract text from the PDF file by calling the function we defined
extracted_text = extract_text_from_pdf(pdf_path)

# Chunk the extracted text by calling the function we defined
text_chunks = chunk_text(extracted_text, 1000, 200)

# Print the number of text chunks created
print("Number of text chunks:", len(text_chunks))

# Print the first text chunk
print("\nFirst text chunk:")
# Print only the first chunk if it exists
if text_chunks:
    print(text_chunks[0])

Number of text chunks: 43

First text chunk:
Understanding Artificial Intelligence  
Chapter 1: Introduction to Artificial Intelligence  
Artificial intelligence (AI) refers to the ability of a digital computer or computer -controlled robot 
to perform tasks commonly associated with intelligent beings. The term is frequently applied to 
the project of developing systems endowed with the intelle ctual processes characteristic of 
humans, such as the ability to reason, discover meaning, generalize, or learn from past 
experience. Over the past few decades, advancements in computing power and data availability 
have significantly accelerated the develo pment and deployment of AI.  
Historical Context  
The idea of artificial intelligence has existed for centuries, often depicted in myths and fiction. 
However, the formal field of AI research began in the mid -20th century. The Dartmouth Workshop 
in 1956 is widely considered the birthplace of AI. Early AI r esearch focused on problem -sol

In [12]:
# --- Imports ---
import os
import google.generativeai as genai
from PyPDF2 import PdfReader

# --- Configure Google API ---
# Get your API key from an environment variable for security
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
if not GOOGLE_API_KEY:
    raise ValueError("GOOGLE_API_KEY not found. Please set the environment variable.")
genai.configure(api_key=GOOGLE_API_KEY)

# --- Function Definitions ---

def extract_text_from_pdf(pdf_path):
    """Extracts text from all pages of a PDF file."""
    text = ""
    with open(pdf_path, 'rb') as file:
        pdf = PdfReader(file)
        for page in pdf.pages:
            text += page.extract_text() or ""
    return text

def chunk_text(text, chunk_size, overlap):
    """Chunks text into segments with a specified overlap."""
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start += chunk_size - overlap
    return chunks

def ask_gemini_about_chunks(chunks, query):
    """
    Uses the Gemini API to answer a query based on a list of text chunks.
    
    Args:
        chunks (list): A list of text strings (chunks).
        query (str): The question to ask the model.
        
    Returns:
        str: The generated answer from the Gemini model.
    """
    model = genai.GenerativeModel('gemini-1.5-flash')
    
    # Create a prompt that includes all the chunks as context
    prompt = f"Using the following text, answer the question: '{query}'\n\nContext:\n" + "\n".join(chunks)
    
    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        return f"An error occurred: {e}"

# --- Main Logic ---

if __name__ == "__main__":
    # Define the path to the PDF file
    pdf_path = "/Users/kekunkoya/Desktop/770 Google /AI_Information.pdf"
    
    # Extract text and chunk it
    extracted_text = extract_text_from_pdf(pdf_path)
    text_chunks = chunk_text(extracted_text, 1000, 200)

    print("Number of text chunks:", len(text_chunks))
    
    if text_chunks:
        print("\nFirst text chunk:")
        print(text_chunks[0][:500], "...")
        
    # Example usage of the Gemini API
    # Define a query to ask the model
    user_query = "What is artificial intelligence?"
    
    # Pass the chunks and the query to the Gemini function
    print(f"\nAsking Gemini: '{user_query}'...")
    gemini_answer = ask_gemini_about_chunks(text_chunks, user_query)
    
    print("\nGemini's Answer:")
    print(gemini_answer)

Number of text chunks: 43

First text chunk:
Understanding Artificial Intelligence  
Chapter 1: Introduction to Artificial Intelligence  
Artificial intelligence (AI) refers to the ability of a digital computer or computer -controlled robot 
to perform tasks commonly associated with intelligent beings. The term is frequently applied to 
the project of developing systems endowed with the intelle ctual processes characteristic of 
humans, such as the ability to reason, discover meaning, generalize, or learn from past 
experience. Over the pa ...

Asking Gemini: 'What is artificial intelligence?'...

Gemini's Answer:
Artificial intelligence (AI) is the ability of a digital computer or computer-controlled robot to perform tasks commonly associated with intelligent beings.  This includes processes characteristic of humans, such as reasoning, discovering meaning, generalizing, and learning from past experience.



## Creating Embeddings for Text Chunks
Embeddings transform text into numerical vectors, which allow for efficient similarity search.

In [13]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key=os.getenv('GOOGLE_API_KEY')

In [14]:
# --- Imports ---
import os
import google.generativeai as genai
from typing import List

# --- Configure Google API ---
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
if not GOOGLE_API_KEY:
    raise ValueError("GOOGLE_API_KEY not found. Please set the environment variable.")
genai.configure(api_key=GOOGLE_API_KEY)

# --- Function Definitions ---

def create_embeddings(text_chunks: List[str], model: str = "models/embedding-001"):
    """
    Creates embeddings for a list of text chunks using the Google Gemini API.

    Args:
        text_chunks (List[str]): A list of text strings to embed.
        model (str): The name of the embedding model to use.

    Returns:
        List[List[float]]: A list of embedding vectors.
    """
    try:
        response = genai.embed_content(
            model=model,
            content=text_chunks
        )
        return response['embedding']
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# --- Main Logic Example ---

if __name__ == "__main__":
    # Define a list of text chunks
    text_chunks = [
        "This is a test chunk about machine learning.",
        "Another piece of text discussing neural networks.",
        "A third chunk about large language models."
    ]

    # Create embeddings for the chunks
    embeddings = create_embeddings(text_chunks)
    
    if embeddings:
        print(f"Created {len(embeddings)} embeddings.")
        print("First embedding vector (snippet):")
        # Print a snippet of the first embedding to show it works
        print(embeddings[0][:5], "...")
    else:
        print("Failed to create embeddings.")

Created 3 embeddings.
First embedding vector (snippet):
[0.016593121, -0.06711799, 0.00490424, -0.004007633, 0.04125118] ...


In [15]:
import os
import google.generativeai as genai

# Configure the Gemini API with your key. It's best to use an environment variable.
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
if not GOOGLE_API_KEY:
    raise ValueError("GOOGLE_API_KEY environment variable not set.")

genai.configure(api_key=GOOGLE_API_KEY)

def create_embeddings(text_chunks, model="models/embedding-001"):
    """
    Creates embeddings for a list of text chunks using the specified Gemini model.

    Args:
        text_chunks (list[str]): The list of input texts for which embeddings are to be created.
        model (str): The Gemini model to be used. Default is "models/embedding-001".

    Returns:
        list: A list of embedding vectors from the Gemini API.
    """
    try:
        response = genai.embed_content(
            model=model,
            content=text_chunks
        )
        # The embeddings are located directly in the 'embedding' key of the response dictionary
        return response['embedding']
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# --- Main Logic ---
if __name__ == "__main__":
    # Define 'text_chunks' as a list of strings
    text_chunks = [
        "This is the first sentence, which we want to embed.",
        "This is the second one, containing different information.",
        "A third text chunk to demonstrate the functionality."
    ]

    # Create embeddings for the text chunks
    embeddings_data = create_embeddings(text_chunks)

    if embeddings_data:
        # Print the number of embeddings created
        print(f"Successfully created {len(embeddings_data)} embeddings.")

        # Print the embedding for the first text chunk (optional)
        print("\nEmbedding for the first chunk (first 5 values):")
        print(embeddings_data[0][:5])

Successfully created 3 embeddings.

Embedding for the first chunk (first 5 values):
[0.04206869, -0.03465894, 0.0016993333, -0.04137134, 0.01707746]


## Performing Semantic Search
We implement cosine similarity to find the most relevant text chunks for a user query.

In [16]:
import numpy as np
import google.generativeai as genai
import os

# --- Gemini API Configuration ---
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
if not GOOGLE_API_KEY:
    raise ValueError("GOOGLE_API_KEY environment variable not set.")
genai.configure(api_key=GOOGLE_API_KEY)

# --- Function Definitions ---

def get_embedding(text: str, model: str = "models/embedding-001") -> np.ndarray:
    """
    Generates an embedding vector for a given text using the Gemini API.

    Args:
        text (str): The input text to embed.
        model (str): The Gemini embedding model to use.

    Returns:
        np.ndarray: The embedding vector as a NumPy array.
    """
    response = genai.embed_content(model=model, content=text)
    return np.array(response['embedding'], dtype=np.float32)

def cosine_similarity(vec1: np.ndarray, vec2: np.ndarray) -> float:
    """
    Calculates the cosine similarity between two vectors.

    Args:
        vec1 (np.ndarray): The first vector.
        vec2 (np.ndarray): The second vector.

    Returns:
        float: The cosine similarity between the two vectors.
    """
    # Compute the dot product and divide by the product of their norms
    dot_product = np.dot(vec1, vec2)
    norm_product = np.linalg.norm(vec1) * np.linalg.norm(vec2)
    return dot_product / norm_product

# --- Main Logic Example ---

if __name__ == "__main__":
    # Get embeddings for two different texts
    text1 = "The cat sat on the mat."
    text2 = "A feline rested on the rug."
    text3 = "The car drove down the street."

    print("Generating embeddings with Gemini...")
    vec1 = get_embedding(text1)
    vec2 = get_embedding(text2)
    vec3 = get_embedding(text3)

    # Calculate and print the cosine similarity between the vectors
    print(f"\nCosine similarity between '{text1}' and '{text2}':")
    similarity_1_2 = cosine_similarity(vec1, vec2)
    print(f"{similarity_1_2:.4f}")

    print(f"\nCosine similarity between '{text1}' and '{text3}':")
    similarity_1_3 = cosine_similarity(vec1, vec3)
    print(f"{similarity_1_3:.4f}")

Generating embeddings with Gemini...

Cosine similarity between 'The cat sat on the mat.' and 'A feline rested on the rug.':
0.8778

Cosine similarity between 'The cat sat on the mat.' and 'The car drove down the street.':
0.6875


In [18]:
def semantic_search(query, text_chunks, embeddings, k=5):
    """
    Performs semantic search on the text chunks using the given query and embeddings.

    Args:
    query (str): The query for the semantic search.
    text_chunks (List[str]): A list of text chunks to search through.
    embeddings (List[dict]): A list of embeddings for the text chunks.
    k (int): The number of top relevant text chunks to return. Default is 5.

    Returns:
    List[str]: A list of the top k most relevant text chunks based on the query.
    """
    # Create an embedding for the query
    query_embedding = create_embeddings(query).data[0].embedding
    similarity_scores = []  # Initialize a list to store similarity scores

    # Calculate similarity scores between the query embedding and each text chunk embedding
    for i, chunk_embedding in enumerate(embeddings):
        similarity_score = cosine_similarity(np.array(query_embedding), np.array(chunk_embedding.embedding))
        similarity_scores.append((i, similarity_score))  # Append the index and similarity score

    # Sort the similarity scores in descending order
    similarity_scores.sort(key=lambda x: x[1], reverse=True)
    # Get the indices of the top k most similar text chunks
    top_indices = [index for index, _ in similarity_scores[:k]]
    # Return the top k most relevant text chunks
    return [text_chunks[index] for index in top_indices]


## Running a Query on Extracted Chunks

In [19]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def semantic_search(query: str, text_chunks: list[str], embeddings: list[list[float]], k: int):
    """
    Performs semantic search using a query, text chunks, and their embeddings.
    """
    # Create an embedding for the query
    query_embedding = create_embeddings(query)[0].embedding

    # Calculate similarity scores between the query and each text chunk
    similarity_scores = cosine_similarity(
        [query_embedding],
        embeddings
    )[0]

    # Get the indices of the top k scores
    top_k_indices = np.argsort(similarity_scores)[-k:][::-1]

    # Return the corresponding text chunks using the full variable name
    #
    # OLD, INCORRECT line:
    # return [text_chunks[i] for i in top_
    #
    # NEW, CORRECT line:
    return [text_chunks[i] for i in top_k_indices]

## Generating a Response Based on Retrieved Chunks

In [20]:
# --- Imports ---
import os
import google.generativeai as genai
# Note: google.generativeai handles its own authentication and exceptions,
# so explicit imports for AuthenticationError are not needed.

# --- 1. Set up your Gemini client ---
# Get your API key from an environment variable
key = os.getenv("GOOGLE_API_KEY")

# Check for API key
if key is None:
    print("Error: GOOGLE_API_KEY environment variable is not set.")
    print("Please set your API key before running the script.")
    exit()

# Configure the genai library with your API key
genai.configure(api_key=key)

# --- 2. Define the generate_response function ---
def generate_response(system_prompt: str, user_message: str, model: str = "gemini-1.5-flash"):
    """
    Generates a response from the Gemini model based on a system prompt and user message.

    Args:
    system_prompt (str): The system prompt to guide the AI's behavior. This will be
                         prepended to the user's message.
    user_message (str): The user's message or query.
    model (str): The Gemini model to be used. Default is "gemini-1.5-flash".

    Returns:
    str: The generated text response from the AI model.
    """
    # The Gemini API handles system prompts and user messages as a single chat history.
    # We combine the system prompt and user message into a single message for simplicity.
    full_prompt = f"{system_prompt}\n\nUser: {user_message}"
    
    try:
        gemini_model = genai.GenerativeModel(model)
        response = gemini_model.generate_content(full_prompt)
        # The text is directly accessible from the response object
        return response.text
    except Exception as e:
        # A more general exception handler is used for Gemini
        print(f"An error occurred during AI response generation: {e}")
        return None

# --- 3. Define the system_prompt and user_prompt variables ---
system_prompt = "You are a helpful and creative AI assistant. Provide concise and relevant answers."
user_prompt = "Tell me a fun fact about the universe."

# --- 4. Generate AI response using the corrected function ---
print("Generating AI response...")
ai_response = generate_response(system_prompt, user_prompt)

# Print the final response content from the AI
if ai_response:
    print("\nAI Response:")
    print(ai_response)

Generating AI response...

AI Response:
There are more stars in the universe than grains of sand on all the beaches on Earth.



## Evaluating the AI Response
We compare the AI response with the expected answer and assign a score.

In [21]:
import os
import google.generativeai as genai

# --- 1. Set up your Gemini client ---
# Load environment variables from a .env file (optional)
# from dotenv import load_dotenv
# load_dotenv()

# Get your API key from the environment variable
key = os.getenv("GOOGLE_API_KEY")

# Basic check for API key
if key is None:
    print("Error: GOOGLE_API_KEY environment variable is not set.")
    print("Please set your API key before running the script.")
    exit()

# Configure the genai library with your API key
genai.configure(api_key=key)

# --- 2. Define the generate_response function ---
def generate_response(system_prompt, user_message, model="gemini-1.5-flash"):
    """
    Generates a response from the Gemini model based on the system prompt and user message.

    Args:
    system_prompt (str): The system prompt to guide the AI's behavior.
    user_message (str): The user's message or query.
    model (str): The Gemini model to be used for generating the response.
                 Default is "gemini-1.5-flash".

    Returns:
    str: The generated text response from the AI model.
    """
    try:
        # Gemini handles the system prompt and user message as a single prompt
        # when using a single turn. For multi-turn conversations, it uses a chat history.
        model = genai.GenerativeModel(model, system_instruction=system_prompt)
        response = model.generate_content(user_message)
        # The text is directly accessible from the response object
        return response.text
    except Exception as e:
        print(f"An error occurred during AI response generation: {e}")
        return None

# --- 3. Define the variables needed for the AI response and evaluation ---
query = "What is the capital of France?"
data = [
    {"question": "What is the capital of France?", "ideal_answer": "Paris"},
    {"question": "Who painted the Mona Lisa?", "ideal_answer": "Leonardo da Vinci"}
]
ai_assistant_system_prompt = "You are a helpful and creative AI assistant. Provide concise and relevant answers."

# --- 4. Generate AI response ---
print(f"User Query: {query}")
try:
    ai_response_content = generate_response(ai_assistant_system_prompt, query)
    if ai_response_content:
        print(f"AI Assistant's Response: {ai_response_content}")
    else:
        ai_response_content = "Error generating AI response."
        print(ai_response_content)
except Exception as e:
    print(f"An error occurred during AI response generation: {e}")
    ai_response_content = "Error generating AI response."


# --- 5. Define the evaluation system prompt ---
evaluate_system_prompt = "You are an intelligent evaluation system. Score AI responses: 1 for 'very close' to the true answer, 0 for 'incorrect/unsatisfactory', and 0.5 for 'partially aligned'. Provide only the score."

# --- 6. Create the evaluation prompt ---
evaluation_prompt = (
    f"User Query: {query}\n"
    f"AI Response:\n{ai_response_content}\n"
    f"True Response: {data[0]['ideal_answer']}\n"
)

# --- 7. Generate the evaluation response ---
print("\nGenerating Evaluation Response...")
try:
    evaluation_score = generate_response(evaluate_system_prompt, evaluation_prompt)
    if evaluation_score:
        print(f"Evaluation Score: {evaluation_score}")
    else:
        print("Error generating evaluation score.")
except Exception as e:
    print(f"An error occurred during evaluation response generation: {e}")

User Query: What is the capital of France?
AI Assistant's Response: Paris


Generating Evaluation Response...
Evaluation Score: 1

